In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report,precision_score, precision_recall_fscore_support as score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import FeatureAgglomeration
from sklearn import cluster
from imblearn.over_sampling import SMOTE
import keras
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical  
import itertools
import keras.backend as K
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

Using TensorFlow backend.


# Word Embeddings and Visualizations Using Gensim

## Embedding with GloVe & Word2Vec

GloVe and Word2Vec are two of the most popular embedding algorithms used in NLP today. Each of these algorithms can be implemented on any set of text data (the bigger the better), and there are two pre-trained and widely used implementations that are easily downloadable. Word2Vec's can be found [here](https://code.google.com/archive/p/word2vec/), and it was trained on a Google News Corpus in 2013. GloVe's can be found [here](https://nlp.stanford.edu/projects/glove/), and it was trained on WikiPedia data in 2014.

All you need to do in order to upload the library of words with their vector representations is download the documents from the links above and parse them as I do below. The libray Gensim makes reading in our embeddings very easy, and it also offers a ton of great features for comparing words. In this analysis we read in the embeddings as words represented as vectors of length 300, but there are other options available.

In [2]:
glove_file = datapath('/Users/JRacanelli/Downloads/glove.6B/glove.6B.300d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")
glove2word2vec(glove_file, tmp_file)
# a dictionary like object containing our words represented as GloVe computed vectors
glove = KeyedVectors.load_word2vec_format(tmp_file)
# a dictionary like object containing our words represented as Word2Vec computed vectors
w2v = gensim.models.KeyedVectors.load_word2vec_format('/Users/JRacanelli/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

## Visualizing our Word Embeddings

Humans understand language as a multitude of associations, and we can think of words as being related to one another. When you read a word like 'enormous' you might begin to think of other words like 'big', or 'gigantic' to help with your understanding. Our brains have established relationships between words, and we can begin to think of words like 'big' and 'enormous' living very "close" to eachother within our language. We can further establish relationships such as "small" being the opposite of big, and having both words in a category of descriptive 'size' words. 

Word2Vec and GloVe are able to infer similar things from large enough amounts of data. Instead of understanding words as we do, our algorithms aim to represent words as vectors, which can be understood as points in a vector space - let's call it our language space. We can now begin to understand where words live in relation to one another. So, continuing our example above, we might be able to find 'big', 'enormous', and 'gigantic' living very close to eachother in our language space. We might further be able to define relationships like 'big' is the opposite of 'small', and both of these words live close to other descriptive size words.

This makes sense in a handwavey sort of way, but we can actually demonstrate this graphically. Our words are now described as vectors of length 300, but we can perform principal component analysis on these vectors and reduce the dimensionality to something we can actually plot. Most of the relationships between our words will, hopefully, be preserved by our PCA. Besides just plotting the points, we can also perform a simple cluster analysis to see how these words are grouped by different clustering algorithms. 

I create a function that takes a set of word embeddings, a list of words, a clustering algorithm, and a specified number of clusters and plots our words (after undergoing PCA) in 2-dimensions. The points are labeled with the words associated, and colored by the clusters that they ended up in. 

In [ ]:
def plot_words(embeddings, words, cluster, n_clusters):

## The Importance of your Input Data

These embeddings are learned from the data that we feed our algorithms. The datasets from Google and Stanford are very different, with Word2Vec's data coming from Google News archives, and GloVe's coming from Wikipedia. The input data completely determines the space of word embeddings, which is why having large and relevant datasets is so important to NLP. Different data sources can come up with wildly different word associations, and we will demonstrate this below.

At the time that these embeddings were computed Donald Trump was not yet president, and had not even mentioned the idea of running. If we check each dataset to see how similar the word `trump` is to the word `president` we will find that they live very far from eachother within our vector space. 

In [10]:
glove.similarity('trump','president')

0.084507439233176246

In [11]:
w2v.similarity('trump','president')

0.029265332098415857

So neither of our datasets were able to make the association between the words `trump` and `president`. That's pretty reasonable; I don't think any of us were able to make that association 5 years ago - perhaps not even the man himself. But what words were associated with the word `trump` in our datasets?

It seems like each dataset had a very different understanding of the word `trump`. GloVe data hones in on Trump the person. This is illustrated by the words associated with him, such as the names of people in his family, as well as words associated with resorts and casinos. Word2Vec, on the other hand, came to associate the word trump with synonyms and derivates of the english word "trump", such as "outweigh" and "trumping". 

While both algorithms produce very different word associations, one can still see how either query returns sensible and interpretable results. The data we feed our algorithms completely determine the mappings that are generated. So, if we were to feed our algorithms more recent news data; our queries would likely yield very different results. We would probably have words like `president`, `election`, and `scandal` living much closer to the word `trump` than it does in either of the examples presented. 

In [3]:
glove.most_similar('trump')

[('ivana', 0.4999052882194519),
 ('melania', 0.45651504397392273),
 ('casino', 0.45222750306129456),
 ('nows', 0.44631344079971313),
 ('knauss', 0.4360748529434204),
 ('hilton', 0.4234514832496643),
 ('trumps', 0.41433754563331604),
 ('ivanka', 0.40609341859817505),
 ('resorts', 0.3992827236652374),
 ('wynn', 0.3902421295642853)]

In [4]:
w2v.most_similar('trump')

[('trumps', 0.7198435068130493),
 ('trumping', 0.580585241317749),
 ('supersede', 0.5600422620773315),
 ('trumped', 0.5497317910194397),
 ('supercede', 0.5309919118881226),
 ('prevail', 0.48776334524154663),
 ('outweigh', 0.4785327613353729),
 ('trample', 0.471425324678421),
 ('overshadow', 0.4701153635978699),
 ('dictate', 0.46754559874534607)]

## How Will this Help my Models?

